https://python.plainenglish.io/scrape-everythings-from-instagram-using-python-39b5a8baf2e5 https://towardsdatascience.com/clustering-documents-with-python-97314ad6a78d

In [9]:
import pandas as pd
from datetime import datetime
from itertools import dropwhile, takewhile
from instagram_web_api import Client, ClientError, ClientLoginError, ClientCookieExpiredError
import string
import random
import hashlib
import getpass
import datetime
import os
import json
import codecs
import re
import pickle

### setup api client

In [2]:
user = input('user: ')
passwrd = getpass.getpass('password: ')
settingsfile = './data/settings.json'

user: jastiso
password: ········


In [3]:
class MyClient(Client):
    @staticmethod
    def _extract_rhx_gis(html):
        options = string.ascii_lowercase + string.digits
        text = ''.join([random.choice(options) for _ in range(8)])
        return hashlib.md5(text.encode())
    
    def login(self):
        """Login to the web site."""
        if not self.username or not self.password:
            raise ClientError('username/password is blank')

        time = str(int(datetime.datetime.now().timestamp()))
        enc_password = f"#PWD_INSTAGRAM_BROWSER:0:{time}:{self.password}"

        params = {'username': self.username, 'enc_password': enc_password, 'queryParams': '{}', 'optIntoOneTap': False}
        self._init_rollout_hash()
        login_res = self._make_request('https://www.instagram.com/accounts/login/ajax/', params=params)
        if not login_res.get('status', '') == 'ok' or not login_res.get ('authenticated'):
            raise ClientLoginError('Unable to login')

        if self.on_login:
            on_login_callback = self.on_login
            on_login_callback(self)
        return login_res
    

In [6]:
api = MyClient(auto_patch=True, authenticate=True, username=user, password=passwrd)

### nyc

In [7]:
data = api.tag_feed('nyctattooartist', count=50)
res = data['data']['hashtag']['edge_hashtag_to_media']['edges']
has_more = data['data']['hashtag']['edge_hashtag_to_media']['page_info']['has_next_page']
end_cursor = data['data']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']
while has_more & (len(res) <= 10000):
        curr_res = api.tag_feed('nyctattoo', count=50, end_cursor=end_cursor)
        has_more = curr_res['data']['hashtag']['edge_hashtag_to_media']['page_info']['has_next_page']
        end_cursor = curr_res['data']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']
        res.extend(curr_res['data']['hashtag']['edge_hashtag_to_media']['edges'])


In [ ]:
f = open('./data/nyc', 'wb')
pickle.load(res, f)

### sf

In [12]:
data = api.tag_feed('sftattooartist', count=50)
res = data['data']['hashtag']['edge_hashtag_to_media']['edges']
has_more = data['data']['hashtag']['edge_hashtag_to_media']['page_info']['has_next_page']
end_cursor = data['data']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']
while has_more & (len(res) <= 10000):
        curr_res = api.tag_feed('sftattooartist', count=50, end_cursor=end_cursor)
        has_more = curr_res['data']['hashtag']['edge_hashtag_to_media']['page_info']['has_next_page']
        end_cursor = curr_res['data']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']
        res.extend(curr_res['data']['hashtag']['edge_hashtag_to_media']['edges'])


In [14]:
f = open('./data/sf', 'wb')
pickle.dump(res, f)